In [1]:
import sys
import json

from pathlib import Path
from dateutil import parser

import requests
import fiona

import pandas as pd
import geopandas as gpd

DIR = Path('..')
sys.path.append(str(DIR))

import pyotp as otp

DATA_DIR = DIR/'data/'

%load_ext autoreload
%autoreload 2

# Palyground

In [2]:
# setting the analysis date and time
dt = parser.parse('25/Oct/2018 8:00:00 AM')

in_gdf = gpd.read_file(str(DATA_DIR/'sample_locations.geojson'))

In [3]:
in_gdf

,stop_id,stop_code,stop_name,stop_lat,stop_lon,geometry
0,1321.0,7427.0,659 Remuera Rd,-36.879865,174.81715,POINT (174.8171499998194 -36.87986499959732)
1,5107.0,7083.0,Mayoral Drive Opposite Aotea Centre,-36.851833,174.76099,POINT (174.760989999634 -36.85183300038418)
2,1843.0,8314.0,464 Sandringham Rd,-36.888577,174.73741,POINT (174.7374099997028 -36.88857699965718)


In [4]:
otp.route(
    locations_gdf = in_gdf.head(2), #a pair of locations in geodataframe fromat
    mode = 'TRANSIT,WALK',
    trip_name = 'test', 
    date_time = dt,
    control_vars = {
        'maxWalkDistance':'500',
        'wheelchair':False,
    },
)

,trip_name,leg_id,mode,from,from_name,to,to_name,route_id,trip_id,distance,duration,startTime,endTime,waitTime,geometry
0,test,0,WALK,POINT (174.8171499998194 -36.87986499959732),Origin,POINT (174.81715 -36.87986),1:7427-20181005121442_v71.5,,,0.192,1,1540407821000,1540407822000,1000,"LINESTRING (17.481712 -3.687981, 17.481712 -3...."
1,test,1,BUS,POINT (174.81715 -36.87986),1:7427-20181005121442_v71.5,POINT (174.76373 -36.85073),1:7086-20181005121442_v71.5,1:07501-20181005121442_v71.5,1:442135527-20181005121442_v71.5,7048.98,2178,1540407823000,1540410001000,1000,"LINESTRING (17.481715 -3.687986, 17.481712 -3...."
2,test,2,WALK,POINT (174.76373 -36.85073),1:7086-20181005121442_v71.5,POINT (174.760989999634 -36.85183300038418),Destination,,,333.405,268,1540410002000,1540410270000,NaN,"LINESTRING (17.476375 -3.685068, 17.476357 -3...."


In [5]:
otp.service_area(
    in_gdf = in_gdf,
    id_field = 'stop_name',
    mode = "BICYCLE", 
    breaks = [500, 1000], #in seconds
    date_time = dt,
    control_vars = {'maxWalkDistance':'400'},
)

{'Access-Control-Allow-Credentials': 'false', 'Content-Type': 'application/x-zip-compressed', 'Date': 'Mon, 21 Jan 2019 03:18:40 GMT', 'Transfer-Encoding': 'chunked'}
{'Access-Control-Allow-Credentials': 'false', 'Content-Type': 'application/x-zip-compressed', 'Date': 'Mon, 21 Jan 2019 03:19:02 GMT', 'Transfer-Encoding': 'chunked'}
{'Access-Control-Allow-Credentials': 'false', 'Content-Type': 'application/x-zip-compressed', 'Date': 'Mon, 21 Jan 2019 03:19:27 GMT', 'Transfer-Encoding': 'chunked'}


,time,geometry,name
0,60000,"POLYGON ((174.457374975947 -36.60744500894256,...",659 Remuera Rd
1,30000,"POLYGON ((174.457374975947 -36.60744500894256,...",659 Remuera Rd
2,60000,POLYGON ((174.5452041682058 -37.02319785620721...,Mayoral Drive Opposite Aotea Centre
3,30000,POLYGON ((174.5452041682058 -37.02319785620721...,Mayoral Drive Opposite Aotea Centre
4,60000,POLYGON ((174.9175795127303 -37.06124647696944...,464 Sandringham Rd
5,30000,POLYGON ((174.9175795127303 -37.06124647696944...,464 Sandringham Rd


In [6]:
od_set = gpd.read_file(str(DATA_DIR/'od_set.geojson'))
od_set['location_name'] = 'location: '+ od_set.index.astype(str)
od_set

,route_id,geometry,location_name
0,1,POINT (174.7298018990001 -36.87621789599996),location: 0
1,1,POINT (174.7639865660001 -36.84994778099997),location: 1
2,2,POINT (174.7647225510001 -36.90041207899998),location: 2
3,2,POINT (174.7271483520001 -36.89949745399997),location: 3
4,3,POINT (174.7085681680001 -36.88324049999994),location: 4
5,3,POINT (174.8049270710001 -36.9133301599999),location: 5
6,4,POINT (174.830714979 -36.89881761199996),location: 6
7,4,POINT (174.835486799 -36.86993589899998),location: 7
8,5,POINT (174.7980088750001 -36.88176517999995),location: 8
9,5,POINT (174.7413818690001 -36.85628036899993),location: 9


In [12]:
otp.od_matrix(
    origins = od_set.head(2),
    destinations = od_set.head(2),
    mode = "TRANSIT,WALK",
    origins_name = 'location_name',
    destinations_name = 'location_name',
    date_time = dt,
    control_vars = {'maxWalkDistance':'400'},
)

Analysis started at: 2019-01-21 17:00:52.800251
{'Access-Control-Allow-Credentials': 'false', 'Content-Type': 'application/x-zip-compressed', 'Date': 'Mon, 21 Jan 2019 04:00:53 GMT', 'Transfer-Encoding': 'chunked'}
{'Access-Control-Allow-Credentials': 'false', 'Content-Type': 'application/x-zip-compressed', 'Date': 'Mon, 21 Jan 2019 04:00:56 GMT', 'Transfer-Encoding': 'chunked'}
remaining origins 1, estimated remaining time: 0:00:03.944358
remaining origins 0, estimated remaining time: 0:00:00
Elapsed time was 0:00:04.965774 seconds


,trip_name,leg_id,mode,from,from_name,to,to_name,route_id,trip_id,distance,duration,startTime,endTime,waitTime,geometry
0,from location: 0 to location: 1,0,WALK,POINT (174.7298018990001 -36.87621789599996),Origin,POINT (174.73232 -36.8756),1:8219-20181005121442_v71.5,,,283.621,224,1540407641000,1540407865000,1000,"LINESTRING (17.47298 -3.687612, 17.472991 -3.6..."
1,from location: 0 to location: 1,1,BUS,POINT (174.73232 -36.8756),1:8219-20181005121442_v71.5,POINT (174.75198 -36.86023),1:8101-20181005121442_v71.5,1:02005-20181005121442_v71.5,1:51437146946-20181005121442_v71.5,2798.79,807,1540407866000,1540408673000,200000,"LINESTRING (17.473232 -3.68756, 17.473236 -3.6..."
2,from location: 0 to location: 1,2,BUS,POINT (174.75198 -36.86023),1:8101-20181005121442_v71.5,POINT (174.76456 -36.8489),1:7096-20181005121442_v71.5,1:19507-20181005121442_v71.5,1:1130146564-20181005121442_v71.5,1861.03,527,1540408873000,1540409400000,1000,"LINESTRING (17.475198 -3.686023, 17.475204 -3...."
3,from location: 0 to location: 1,3,WALK,POINT (174.76456 -36.8489),1:7096-20181005121442_v71.5,POINT (174.7639865660001 -36.84994778099997),Destination,,,150.682,129,1540409401000,1540409530000,NaN,"LINESTRING (17.476456 -3.684889, 17.476441 -3...."
4,from location: 1 to location: 0,0,WALK,POINT (174.7639865660001 -36.84994778099997),Origin,POINT (174.76378 -36.84869),1:1362-20181005121442_v71.5,,,201.947,168,1540408151000,1540408319000,1000,"LINESTRING (17.476398 -3.684996, 17.476398 -3...."
5,from location: 1 to location: 0,1,BUS,POINT (174.76378 -36.84869),1:1362-20181005121442_v71.5,POINT (174.74647 -36.86479),1:8106-20181005121442_v71.5,1:01804-20181005121442_v71.5,1:1080127665-20181005121442_v71.5,2454.82,520,1540408320000,1540408840000,138000,"LINESTRING (17.476378 -3.684869, 17.476382 -3...."
6,from location: 1 to location: 0,2,BUS,POINT (174.74647 -36.86479),1:8106-20181005121442_v71.5,POINT (174.73198 -36.87597),1:8218-20181005121442_v71.5,1:02006-20181005121442_v71.5,1:51437147042-20181005121442_v71.5,2055.66,567,1540408978000,1540409545000,1000,"LINESTRING (17.474647 -3.686479, 17.474645 -3...."
7,from location: 1 to location: 0,3,WALK,POINT (174.73198 -36.87597),1:8218-20181005121442_v71.5,POINT (174.7298018990001 -36.87621789599996),Destination,,,260.254,207,1540409546000,1540409753000,NaN,"LINESTRING (17.473193 -3.687593, 17.473204 -3...."
